# This notebook demos a diffusion stencil

We start with some imports & a small helper function to convert our stencil to SIR:

In [8]:
from typing import Callable

from inspect import getsource
import ast

from dusk.grammar import Grammar

from dawn4py import compile, CodeGenBackend
from dawn4py.serialization import make_sir, SIR
from dawn4py.serialization.SIR import GridType
from dawn4py._dawn4py import run_optimizer_sir


def dusk_to_sir(stencil: Callable) -> SIR:
    # this will give wrong line numbers, there should be a way to fix them
    name = stencil.__name__
    stencil = ast.parse(getsource(stencil))

    assert isinstance(stencil, ast.Module)
    assert len(stencil.body) == 1
    stencil = stencil.body[0]
    assert Grammar.is_stencil(stencil)

    return make_sir(
        name, GridType.Value("Unstructured"), [Grammar().stencil(stencil)]
    )

With the above definitions, we can write our diffusion stencil:

In [9]:
from dusk.script import *


@stencil
def diffusion(
    TV: Field[Vertex],
    TE: Field[Edge], TEinit: Field[Edge], TE_t: Field[Edge], TEnabla2: Field[Edge],
    inv_primal_edge_length: Field[Edge], inv_vert_vert_length: Field[Edge], nnbhV: Field[Vertex],
    boundary_edge: Field[Edge],
    kappa: Field[Edge], dt: Field[Edge]
) -> None:

    with levels_upward:
        #TODO replace with diffusion solver
        pass    


Then we can use dusk to convert the stencil to SIR. With dawn we can compile SIR to C++ which we will write to `diffusion_cxx-naive.cpp`:

In [10]:
sir = dusk_to_sir(diffusion)
cpp_naive = compile(sir, backend=CodeGenBackend.CXXNaiveIco)
with open("diffusion_cxx-naive.cpp", "w+") as f:
    f.write(cpp_naive)

The generated C++ code also requires a driver which is already setup for this demo. With the driver code we can generate an executable `runner`:

In [4]:
!make

First, we put the runner into test mode to ensure that the computed kernel is correct:

In [5]:
!./runner test

mesh stats: #cells 680 #nodes 378 #edges 1057
congratulations!, your stencil is correct, you can visualize now!

If the tester reported that your dusk stencil works correctly, you can now run the complete stencil and visualize

In [6]:
!./runner run

mesh stats: #cells 680 #nodes 378 #edges 1057
time 0.105 timestep 1 dt 0.005
time 0.11 timestep 2 dt 0.005
time 0.115 timestep 3 dt 0.005
time 0.12 timestep 4 dt 0.005
time 0.125 timestep 5 dt 0.005
time 0.13 timestep 6 dt 0.005
time 0.135 timestep 7 dt 0.005
time 0.14 timestep 8 dt 0.005
time 0.145 timestep 9 dt 0.005
time 0.15 timestep 10 dt 0.005
time 0.155 timestep 11 dt 0.005
time 0.16 timestep 12 dt 0.005
time 0.165 timestep 13 dt 0.005
time 0.17 timestep 14 dt 0.005
time 0.175 timestep 15 dt 0.005
time 0.18 timestep 16 dt 0.005
time 0.185 timestep 17 dt 0.005
time 0.19 timestep 18 dt 0.005
time 0.195 timestep 19 dt 0.005
time 0.2 timestep 20 dt 0.005
time 0.205 timestep 21 dt 0.005
time 0.21 timestep 22 dt 0.005
time 0.215 timestep 23 dt 0.005
time 0.22 timestep 24 dt 0.005
time 0.225 timestep 25 dt 0.005
time 0.23 timestep 26 dt 0.005
time 0.235 timestep 27 dt 0.005
time 0.24 timestep 28 dt 0.005
time 0.245 timestep 29 dt 0.005
time 0.25 timestep 30 dt 0.005
time 0.255 timestep

In [9]:
%%capture
import prep_animation as pa
import matplotlib.animation as animation
from IPython.display import HTML
animate = animation.FuncAnimation(pa.fig, pa.update_plot, len(pa.out_files), fargs=(pa.zarray, pa.plot), interval = 50)

In [10]:
HTML(animate.to_html5_video())